In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as mp
import seaborn as sns
%matplotlib inline

In [15]:
# EJ(votos) + Mautic(contatos) + Dados gerais de Analytics
df1 = pd.read_csv('/tmp/airflow/ej_analytics_mautic.csv',dtype={'analytics_client_id': str})
#sns.countplot(x="analytics_source", hue="escolha", data=df1)
df1.dropna()['analytics_client_id'].value_counts()

1952709213.1586711979    24
1687884283.1544397112    16
1935361641.1588287568     5
Name: analytics_client_id, dtype: int64